In [1]:
cd ..

/home/cody/inner-knowledge-pruning


In [2]:
from fastai.vision.all import *
from fastai.distributed import *
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms

from src.dataloader import make_data_loader

In [3]:
dataloaders, _ = make_data_loader(32, 'imagenette')

In [4]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7faeb5214a60>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7faeb520e970>}

In [5]:


dls = DataLoaders(dataloaders['train'], dataloaders['val'])

In [6]:
model = models.vgg16_bn(pretrained=False)

In [7]:
model.classifier[-1] = nn.Linear(in_features=4096, out_features=10)
model = model.cuda()

In [8]:
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy, wd=0.01, cbs=[CudaCallback, SaveModelCallback()], path=Path.cwd()/'tmp_cifar10')
learn.model = learn.model.cuda()

In [9]:
learn.fit_one_cycle(240, 4e-4, wd=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.063437,1.880129,0.394140,01:18
1,1.884375,1.689184,0.507261,01:17
2,1.756509,1.559389,0.546752,01:17
3,1.648919,1.414226,0.613503,01:17
4,1.567132,1.380008,0.641783,01:18
5,1.489660,1.372251,0.631083,01:18
6,1.461759,1.393465,0.627771,01:18
7,1.439752,1.414797,0.638471,01:18
8,1.406650,1.410941,0.629045,01:18
9,1.392467,1.304917,0.670573,01:18


Better model found at epoch 0 with valid_loss value: 1.880129098892212.
Better model found at epoch 1 with valid_loss value: 1.6891835927963257.
Better model found at epoch 2 with valid_loss value: 1.5593888759613037.
Better model found at epoch 3 with valid_loss value: 1.4142262935638428.
Better model found at epoch 4 with valid_loss value: 1.380008339881897.
Better model found at epoch 5 with valid_loss value: 1.372251272201538.
Better model found at epoch 9 with valid_loss value: 1.3049170970916748.
Better model found at epoch 12 with valid_loss value: 1.2168267965316772.
Better model found at epoch 15 with valid_loss value: 1.128006100654602.
Better model found at epoch 19 with valid_loss value: 1.0706497430801392.
Better model found at epoch 29 with valid_loss value: 1.017169713973999.
Better model found at epoch 37 with valid_loss value: 0.9994684457778931.
Better model found at epoch 39 with valid_loss value: 0.983372688293457.
Better model found at epoch 40 with valid_loss valu

In [10]:
learn.validate()

Better model found at epoch 0 with valid_loss value: 0.9375796318054199.


(#2) [0.6599047183990479,0.9375796318054199]

learn.fit_one_cycle(120, 1.5e-3, wd=1e-3) : [0.7092109322547913,0.9151592254638672]

learn.fit_one_cycle(120, 4e-4, wd=1e-3) : [0.6876890063285828,0.9276432991027832]

learn.fit_one_cycle(120, 1e-4, wd=1e-3): [0.761014461517334,0.9164330959320068]


In [14]:
learn.save(Path().cwd()/'./models/imagenette/vgg16')

Path('/home/cody/inner-knowledge-pruning/models/imagenette/vgg16.pth')